In [20]:
import xml.etree.ElementTree as ET
import os
import xmltodict
import torch
from torch.utils.data import Dataset
import transformers as T
import shutil
import random
from PIL import Image
import numpy as np

ModuleNotFoundError: No module named 'xmltodict'

In [107]:

def xml_handling(root='./archive', target_root='data'):
    root='./archive'
    target_root='data'
    root_label=root+'/annotations'
    root_img=root+'/images/'
    
    xml_files=os.listdir(root_label)    
    i=0
    for file in xml_files:
        if file[0]=='.':
            continue
        file=root_label+"/"+file
        train=False
        if random.random()<0.8:
            train=True
        tree = ET.parse(file)
        root = tree.getroot()
        base_name=os.path.basename(file)
        
        base_name,ext=os.path.splitext(base_name)
        result = ET.tostring(root,encoding='utf-8').decode('utf-8')
        dict_data=xmltodict.parse(result)
        data=dict_data['annotation']['object']
        if len(data)==2:
            continue
        src_img=root_img+base_name+'.png'
        img_path,label_path="",""
        if data['name']=='cat':
            note='Cat'+" "+str(data['bndbox']['xmin'])+" "+str(data['bndbox']['ymin'])+" "+str(data['bndbox']['xmax'])+" "+str(data['bndbox']['ymax'])
            if train:
                label_path=target_root+'/train/Cat/Label/'+base_name+'.txt'
                img_path=target_root+'/train/Cat/'+base_name+'.png'
            else:
                label_path=target_root+'/val/Cat/Label/'+base_name+'.txt'
                img_path=target_root+'/val/Cat/'+base_name+'.png'
        else:
            note='Dog'+" "+str(data['bndbox']['xmin'])+" "+str(data['bndbox']['ymin'])+" "+str(data['bndbox']['xmax'])+" "+str(data['bndbox']['ymax'])
            
            if train:
                label_path=target_root+'/train/Dog/Label/'+base_name+'.txt'
                img_path=target_root+'/train/Dog/'+base_name+'.png'
            else:
                
                label_path=target_root+'/val/Dog/Label/'+base_name+'.txt'
                img_path=target_root+'/val/Dog/'+base_name+'.png'
        shutil.copy2(src_img,img_path)
        print(label_path)
        with open(label_path,'w') as f:
            f.write(note)
        
        


In [1]:
"""class AnimalData:
    def __init__(self,root,transforms):
        self.root=root
        self.transforms=transforms
        classes=sort(list(os.listdir(root)))
        images=[]
        for c in classes:
            if c=='Label':
                continue
            for img in sort(list(os.listdir(c))):
                images.append(root+c+img)
        self.images=images
        label_dir=[c+'/Label' for c in classes]
        labels=[]
        for c in classes:
            for label in sort(list(os.listdir(label_dir))):
                labels.append(root+label_dir+label)
        self.labels=labels
    def __getitem__(self,idx):
        img =Image.open(self.img[idx]).convert("RGB")
        with open(self.labels,'r') as f:
            content= f.read()
            lines=contents.split(' ')
            for i in range(1, len(lines)):
                lines[i]=int(lines[i])
        
        """
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#from bs4 import BeautifulSoup
import torchvision
from torchvision import transforms, datasets, models
import torch
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import matplotlib.patches as patches
import tqdm 
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import random
import os

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

c:\Users\Pc\anaconda3\envs\deeplearning\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_labels=[]
test_labels=[]
train_imgs=[]
test_imgs=[]
classes=['Bear', 'Bull', 'Cat', 'Cattle',
           'Cheetah', 'Chicken', 'Crocodile',
           'Deer', 'Dog', 'Fox', 'Goat', 'Hippopotamus', 
           'Horse', 'Jaguar', 'Leopard', 'Lion', 'Lynx',
           'Monkey', 'Mule', 'Ostrich', 'Owl', 
           'Pig', 'Raccoon', 'Raven', 'Rhinoceros',
           'Sheep', 'Snake', 'Tiger', 'Turkey', 'Turtle', 'Zebra']

"""
classes=['Cat','Dog']"""
num_classes=len(classes)
encode={}
for i in range(len(classes)):
    encode[classes[i]]=i+1

def checkclass(dirname):
    
    for animal in classes:
        if dirname.endswith(animal):
            return True
    return False

for dirname, dirs, filenames in os.walk('./data/'):
    
    if 'train' in dirname and checkclass(dirname):
        labels= dirname +'/Label/'
      
        for label in sorted(os.listdir(labels)):
           
            if  label.endswith('txt'):
                train_labels.append(labels+label)
      
        for image in sorted(filenames):
            
            if image.endswith('png') or image.endswith('jpg'):
                train_imgs.append(dirname+'/'+image)
    if 'val' in dirname and checkclass(dirname):
        labels= dirname +'/Label/'
        for label in sorted(os.listdir(labels)):
            if label.endswith('.txt'):
                test_labels.append(labels+label)
        for image in sorted(filenames):
            if image.endswith('png') or image.endswith('jpg'):
                test_imgs.append(dirname+'/'+image)


In [3]:
print(len(train_labels))
print(len(train_imgs))
print(len(test_imgs))
print(len(test_labels))

14620
14620
6864
6864


In [4]:
class AnimalDataset(object):
    def __init__(self, transforms,img_dir,label_dir):
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = img_dir
        self.label=label_dir
#         self.labels = list(sorted(os.listdir("/kaggle/input/face-mask-detection/annotations/")))

    def __getitem__(self, idx):
        # load images ad masks
        file_image = self.imgs[idx]
        file_label = self.label[idx]
        img = Image.open(file_image).convert("RGB")
        target={}
        target['image_id']=torch.tensor([idx])
        labels=[]
        boxes=[]
        #Generate Label
        with open(file_label,'r')as f:
            lines=f.readlines()
            for line in lines:
                line_list=line.strip().split(' ')
                labels.append(encode[line_list[0]])
                boxes.append([float(i) for i in line_list[1:]])
                
            if self.transforms is not None:
                img=self.transforms(img)
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # Labels (In my case, I only one class: target class or background)
        labels = torch.as_tensor(labels, dtype=torch.int64)        
        
        
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        
        iscrowd = torch.zeros((len(classes),), dtype=torch.int64)

        target['boxes']=boxes
        target['labels']=labels
       
        target["area"] = area
        target["iscrowd"] = iscrowd
        return img, target

    def __len__(self):
        return len(self.imgs)

In [5]:

class AugmentImageBboxes(object):
    def __init__(self):
        pass
    
    def __call__(self, image, bboxes):
        # decide if flip or rotate
        if random.random() < 0.5:
            # Flip
            image = transforms.functional.hflip(image)
            h,w=image.shape[-2:]
            bboxes[:, [0, 2]] = h - bboxes[:, [2, 0]] 
        else:
            # Rotate by 90 degrees
            image = transforms.functional.rotate(image, 90)
            h, w = image.shape[-2:]
            bboxes[:, [0, 1, 2, 3]] = [w - bboxes[:, 3], bboxes[:, 0], h - bboxes[:, 1], w - bboxes[:, 0]]
        return image, bboxes

In [6]:

data_transform = transforms.Compose([
        transforms.ToTensor(),
    ])

In [7]:
def collate_fn(batch):
    return tuple(zip(*batch))

dataset = AnimalDataset(data_transform,train_imgs[:10000],train_labels[:10000])
dataset[0]

(tensor([[[0.5647, 0.5686, 0.5725,  ..., 0.3059, 0.3098, 0.3098],
          [0.5686, 0.5725, 0.5765,  ..., 0.2980, 0.3020, 0.3098],
          [0.5608, 0.5686, 0.5765,  ..., 0.2941, 0.3059, 0.3059],
          ...,
          [0.8824, 0.9255, 0.9608,  ..., 0.9843, 0.9843, 0.9608],
          [0.9176, 0.9176, 0.9333,  ..., 0.9843, 0.9725, 0.9725],
          [0.9490, 0.9373, 0.9412,  ..., 0.9804, 0.9725, 0.9765]],
 
         [[0.5216, 0.5255, 0.5255,  ..., 0.2039, 0.2078, 0.2078],
          [0.5216, 0.5255, 0.5294,  ..., 0.2078, 0.2118, 0.2078],
          [0.5137, 0.5216, 0.5294,  ..., 0.2078, 0.2157, 0.2157],
          ...,
          [0.7647, 0.8078, 0.8471,  ..., 0.8627, 0.8627, 0.8392],
          [0.7843, 0.7843, 0.8000,  ..., 0.8549, 0.8471, 0.8471],
          [0.8157, 0.8039, 0.8078,  ..., 0.8510, 0.8471, 0.8510]],
 
         [[0.4353, 0.4392, 0.4392,  ..., 0.1529, 0.1569, 0.1569],
          [0.4353, 0.4392, 0.4431,  ..., 0.1529, 0.1569, 0.1569],
          [0.4275, 0.4353, 0.4353,  ...,

In [8]:


# load a model pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes =len(classes)+1 # 1 class (person) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


In [9]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
data_loader = torch.utils.data.DataLoader(
 dataset, batch_size=16, collate_fn=collate_fn)
for imgs, annotations in data_loader:
    imgs = list(img.to(device) for img in imgs)
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    print(annotations)
    break
print (device)

[{'image_id': tensor([0], device='cuda:0'), 'boxes': tensor([[192.7997,  81.5522, 848.0000, 605.2450]], device='cuda:0'), 'labels': tensor([1], device='cuda:0'), 'area': tensor([343123.6562], device='cuda:0'), 'iscrowd': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0], device='cuda:0')}, {'image_id': tensor([1], device='cuda:0'), 'boxes': tensor([[389.1200, 148.0047, 800.6400, 649.6826]], device='cuda:0'), 'labels': tensor([1], device='cuda:0'), 'area': tensor([206450.4688], device='cuda:0'), 'iscrowd': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0], device='cuda:0')}, {'image_id': tensor([2], device='cuda:0'), 'boxes': tensor([[  12.8000,    0.0000, 1022.5776,  482.5766]], device='cuda:0'), 'labels': tensor([1], device='cuda:0'), 'area': tensor([487295.0312], device='cuda:0'), 'iscrowd': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [12]:
from engine import train_one_epoch, evaluate
import utils
test_data=AnimalDataset(data_transform,test_imgs[:10],test_labels[:10])
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person


# define training and validation data loaders
train_loader = torch.utils.data.DataLoader(
    dataset, batch_size=16, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

# get the model using our helper function


# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size=3,
                                                gamma=0.1)

# let's train it for 10 epochs
num_epochs = 10

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model,test_loader, device=device)


0it [00:00, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.56 GiB (GPU 0; 8.00 GiB total capacity; 6.19 GiB already allocated; 0 bytes free; 6.97 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [42]:

test_data=AnimalDataset(data_transform,test_imgs[:10],test_labels[:10])

test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)
evaluate(model,test_loader, device=device)

creating index...
index created!


In [11]:
torch.cuda.empty_cache()

In [ ]:
ssdmodel=